In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import *
import warnings
import matplotlib.pyplot as plt
import pickle
print("tf.__version__ is:", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)
warnings.filterwarnings('ignore')
%matplotlib inline

Using TensorFlow backend.


tf.__version__ is: 2.2.0
tf.keras.__version__ is: 2.3.0-tf


In [2]:
### Read Dataset
df = pd.read_csv("train_final.csv", index_col=False)
labels = df[['784']]

In [3]:
df.drop(df.columns[[784]], axis=1, inplace=True)
df.head()

0    1    2    3    4    5    6    7    8    9  ...  774  775  776  777  \
0    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1  255  255  255  255  255  255  255  255  255  255  ...    0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   778  779  780  781  782  783  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 784 columns]

In [4]:
labels = np.array(labels)

In [5]:
### Some variables
VERBOSES = 1
EPOCHS = 10
NUM_CLASSES = 13
BATCH_SIZE = 500

In [6]:
import keras.backend.tensorflow_backend as tfback
from keras import backend as K
K.common.set_image_dim_ordering('th')

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

In [7]:
from keras.utils.np_utils import to_categorical
labels = to_categorical(labels, num_classes=NUM_CLASSES)
print(labels[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [8]:
len(df)

47504

In [9]:
dataset=list()
for i in range(len(df)):
    dataset.append(np.array(df[i:i+1]).reshape(1, 28, 28))
dataset = np.array(dataset)

In [10]:
np.random.seed(7)

In [11]:
model = Sequential()

In [12]:
### Adding Layers
model.add(Convolution2D(64, (5, 5), input_shape=(1, 28, 28), activation = 'relu', data_format = 'channels_first'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Convolution2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.33))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(NUM_CLASSES, activation = 'softmax'))

# Compiling our model
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 24, 24)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 12, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 10, 32)        6944      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 5, 32)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 5, 32)         128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 5, 32)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2400)             

In [13]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(dataset, labels, random_state=100, test_size=0.33)

In [14]:
model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    verbose=VERBOSES
)

Train on 42753 samples, validate on 4751 samples
Epoch 1/10
42753/42753 [==============================] - 73s 2ms/step - loss: 0.4772 - accuracy: 0.8565 - val_loss: 1.3091 - val_accuracy: 0.6809
Epoch 2/10
42753/42753 [==============================] - 76s 2ms/step - loss: 0.0856 - accuracy: 0.9770 - val_loss: 0.1129 - val_accuracy: 0.9674
Epoch 3/10
42753/42753 [==============================] - 52s 1ms/step - loss: 0.0433 - accuracy: 0.9892 - val_loss: 0.0433 - val_accuracy: 0.9909
Epoch 4/10
42753/42753 [==============================] - 77s 2ms/step - loss: 0.0286 - accuracy: 0.9924 - val_loss: 0.0323 - val_accuracy: 0.9945
Epoch 5/10
42753/42753 [==============================] - 81s 2ms/step - loss: 0.0180 - accuracy: 0.9955 - val_loss: 0.0351 - val_accuracy: 0.9909
Epoch 6/10
42753/42753 [==============================] - 75s 2ms/step - loss: 0.0129 - accuracy: 0.9964 - val_loss: 0.0292 - val_accuracy: 0.9941
Epoch 7/10
42753/42753 [==============================] - 70s 2ms/ste

In [16]:
model_json = model.to_json()
model_name = "final_model"
with open("../models/{}.json".format(model_name), "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("../models/{}.h5".format(model_name))
print("Model saved as `{}`".format(model_name))

Model saved as final_model
